In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('open_pubs.csv.gz')
df.count()

fas_id             51566
name               51566
address            51566
postcode           51566
easting            51566
northing           51566
latitude           51566
longitude          51566
local_authority    51564
dtype: int64

In [3]:
df['district'],df['sector'] = df['postcode'].str.split(' ').str

In [4]:
df_2 = df.query('district=="SE5"')
df_2.head()

,fas_id,name,address,postcode,easting,northing,latitude,longitude,local_authority,district,sector
12066,124266,The Cambria Bar & Brasserie,"40 Kemerton Road, London",SE5 9AR,532292,175699.0,51.464809,-0.096958,Lambeth,SE5,9AR
12084,124336,The Kennington,"60 Camberwell New Road, London",SE5 0RS,531624,177275.0,51.479119,-0.105983,Lambeth,SE5,0RS
12118,124416,The Sun Of Camberwell,"61 - 63 Coldharbour Lane, London",SE5 9NS,532361,176242.0,51.469682,-0.095762,Lambeth,SE5,9NS
12587,133358,Amaryllis,"66, Coldharbour Lane, London",SE5 9PU,532275,176249.0,51.469756,-0.096997,Lambeth,SE5,9PU
12601,133614,British queen,"34, Picton Street, London",SE5 7QH,532819,177190.0,51.478076,-0.088817,Southwark,SE5,7QH


In [5]:
df_3=df_2.copy()
df_3[['latitude','longitude']]=df_2[['latitude','longitude']].apply(pd.to_numeric, errors='coerce').dropna()
df_3.head()

,fas_id,name,address,postcode,easting,northing,latitude,longitude,local_authority,district,sector
12066,124266,The Cambria Bar & Brasserie,"40 Kemerton Road, London",SE5 9AR,532292,175699.0,51.464809,-0.096958,Lambeth,SE5,9AR
12084,124336,The Kennington,"60 Camberwell New Road, London",SE5 0RS,531624,177275.0,51.479119,-0.105983,Lambeth,SE5,0RS
12118,124416,The Sun Of Camberwell,"61 - 63 Coldharbour Lane, London",SE5 9NS,532361,176242.0,51.469682,-0.095762,Lambeth,SE5,9NS
12587,133358,Amaryllis,"66, Coldharbour Lane, London",SE5 9PU,532275,176249.0,51.469756,-0.096997,Lambeth,SE5,9PU
12601,133614,British queen,"34, Picton Street, London",SE5 7QH,532819,177190.0,51.478076,-0.088817,Southwark,SE5,7QH


In [6]:
df_4=df_3.sample(n=10,random_state=42).reset_index()
df_4.head(8)

,index,fas_id,name,address,postcode,easting,northing,latitude,longitude,local_authority,district,sector
0,12066,124266,The Cambria Bar & Brasserie,"40 Kemerton Road, London",SE5 9AR,532292,175699.0,51.464809,-0.096958,Lambeth,SE5,9AR
1,12653,135094,Old dispensary,"325, Camberwell New Road, London",SE5 0TF,532319,176888.0,51.475479,-0.096125,Southwark,SE5,0TF
2,12636,134393,Hermits cave,"28, Camberwell Church Street, London",SE5 8QU,532655,176711.0,51.473810,-0.091357,Southwark,SE5,8QU
3,12084,124336,The Kennington,"60 Camberwell New Road, London",SE5 0RS,531624,177275.0,51.479119,-0.105983,Lambeth,SE5,0RS
4,12687,135575,St georges tavern,"14, Coleman Road, London",SE5 7TG,533187,177341.0,51.479346,-0.083464,Southwark,SE5,7TG
5,12612,133843,Clarendon arms,"225, Camberwell New Road, London",SE5 0TH,532003,177089.0,51.477359,-0.100597,Southwark,SE5,0TH
6,12760,135954,The planet nollywood,"Father Redcap, 319, Camberwell Road, London",SE5 0HQ,532498,176944.0,51.475940,-0.093528,Southwark,SE5,0HQ
7,12642,134552,Joiners arms,"35, Denmark Hill, London",SE5 8RS,532565,176474.0,51.471710,-0.092740,Southwark,SE5,8RS


In [7]:
sample_locations = df_4[['latitude','longitude']].as_matrix()
sample_locations

array([[ 51.464809,  -0.096958],
       [ 51.475479,  -0.096125],
       [ 51.47381 ,  -0.091357],
       [ 51.479119,  -0.105983],
       [ 51.479346,  -0.083464],
       [ 51.477359,  -0.100597],
       [ 51.47594 ,  -0.093528],
       [ 51.47171 ,  -0.09274 ],
       [ 51.469756,  -0.096997],
       [ 51.478076,  -0.088817]])

In [8]:
#Example 2-opt algorithm found here:
#https://stackoverflow.com/a/44080908/4537686

path_distance_lambda = lambda r,c: np.sum([np.linalg.norm(c[r[p]]-c[r[p-1]]) for p in range(len(r))])
#Repurose lamba function in function for easier debugging.
def path_distance(route,cities):
    route_path_lengths=[]
    for p in route:
        route_path_lengths.append(np.linalg.norm(cities[route[p]]-cities[route[p-1]]))
    return np.sum(route_path_lengths)

# Reverse the order of all elements from element i to element k in array r.
two_opt_swap_lambda = lambda r,i,k: np.concatenate((r[0:i],r[k:-len(r)+i-1:-1],r[k+1:len(r)]))

def two_opt(cities,improvement_threshold): # 2-opt Algorithm adapted from https://en.wikipedia.org/wiki/2-opt
    route = np.arange(cities.shape[0]) # Make an array of row numbers corresponding to cities.
    improvement_factor = 1 # Initialize the improvement factor.
    best_distance = path_distance(route,cities) # Calculate the distance of the initial path.
    while improvement_factor > improvement_threshold: # If the route is still improving, keep going!
        distance_to_beat = best_distance # Record the distance at the beginning of the loop.
        for swap_first in range(1,len(route)-2): # From each city except the first and last,
            for swap_last in range(swap_first+1,len(route)): # to each of the cities following,
                new_route = two_opt_swap_lambda(route,swap_first,swap_last) # try reversing the order of these cities
                new_distance = path_distance(new_route,cities) # and check the total distance with this modification.
                if new_distance < best_distance: # If the path distance is an improvement,
                    route = new_route # make this the accepted best route
                    best_distance = new_distance # and update the distance corresponding to this route.
        improvement_factor = 1 - best_distance/distance_to_beat # Calculate how much the route has improved.
    return route # When the route is no longer improving substantially, stop searching and return the route.

In [9]:
route = two_opt(sample_locations,0.001)

In [10]:
import matplotlib.pyplot as plt
# Reorder the cities matrix by route order in a new matrix for plotting.
new_location_order = np.concatenate((np.array([sample_locations[route[i]] for i in range(len(route))]),np.array([sample_locations[0]])))

In [11]:
import folium

lat_max = df_4['latitude'].max()
lat_min = df_4['latitude'].min()
lon_max = df_4['longitude'].max()
lon_min = df_4['longitude'].min()

m = folium.Map()
m.fit_bounds([[lat_max, lon_max], [lat_min, lon_min]])

points_arr=new_location_order.tolist()

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.Marker(location=[point[0], point[1]]).add_to(m)
    
for point in points_arr:
    plotDot(point)

folium.PolyLine(points_arr).add_to(m)
m

In [82]:
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

two_dimentional_location_array = df_4[['latitude','longitude']].values
sf = squareform(pdist(two_dimentional_location_array))

In [83]:
df_5=pd.DataFrame(sf, columns=df_4.fas_id.unique(), index=df_4.fas_id.unique())

In [84]:
list(df_5.columns.values)

[345564, 342250, 342143, 345289, 342150, 345524, 345411, 343739]